## 0) Initialization

### 0.1) Configure environment and import dependencies

In [ ]:
# Standard library imports
import os
import sys
import csv
import shutil
import importlib
from functools import reduce
from typing import List, Dict
import glob

# Third-party imports
import pandas as pd
import yaml

# Add repository root to PYTHONPATH to enable absolute imports of local modules
PROJECT_ROOT = os.path.abspath(os.getcwd())
sys.path.insert(0, PROJECT_ROOT)


### 0.2) Define paths, constants and create output directories

In [ ]:
# Load configuration
with open(os.path.join(PROJECT_ROOT, "config", "config_feature_extraction.yaml"), "r") as f:
    cfg = yaml.safe_load(f)

# Paths
EVENT_LOG          = cfg["event_log"]
EVENT_LOG_SETTINGS = cfg["event_log_settings"]
exp_name           = cfg["experiment_name"]
RAW_LOG            = os.path.join(PROJECT_ROOT, "0_raw_log", cfg["raw_log"])
UNIQUE_DIR         = os.path.join(PROJECT_ROOT, "1_unique_log")
LABELED_DIR        = os.path.join(PROJECT_ROOT, "2_labelled_logs", EVENT_LOG)
FEATURES_DIR       = os.path.join(PROJECT_ROOT, "3_extracted_features", EVENT_LOG)

# Processing parameters
SPLIT_RATIO   = 0.7  # fraction of events used for training
SEQ_THRESHOLD = 5    # minimum number of events required per sequence
MAX_SPLITS    = 1    # number of train/test splits

# Ensure output directories exist
os.makedirs(UNIQUE_DIR, exist_ok=True)
os.makedirs(LABELED_DIR, exist_ok=True)
os.makedirs(FEATURES_DIR, exist_ok=True)

# Summary
print(f"Event Log: {EVENT_LOG}")
print(f"Experiment: {exp_name}")

### 0.3) Load modules for log preparation, tagging, splitting, and feature extraction

In [ ]:
# Log preparation and tagging utilities
from helper_functions.preparation.RetagLogWithUniqueIds         import changeLog
from helper_functions.preparation.TaggingStrategy               import TaggingStrategy
from helper_functions.preparation.ConfigurationFile             import ConfigurationFile
from helper_functions.preparation.PayloadType                   import PayloadType
from helper_functions.preparation.declaretemplates_new          import template_response, template_exist
from helper_functions.preparation.LogTaggingViaPredicates       import (
    tagLogWithOccurrence,
    tagLogWithValueEqOverEventAttn,
    tagLogWithSatAllProp,
    tagLogWithExactOccurrence,
    tagLogWithValueEqOverIthEventAttn,
    SatCases
)

# Log splitting and feature extraction helpers
from helper_functions.feature_extraction.deviancecommon         import read_XES_log
from helper_functions.feature_extraction.FileNameUtils          import csv_trace_encodings
from helper_functions.feature_extraction.TraceUtils             import getTraceId
from helper_functions.feature_extraction.LogUtils               import (
    xes_to_propositional_split,
    xes_to_data_propositional_split,
    xes_to_tracelist_split
)

# Feauture extraction helpers
from helper_functions.feature_extraction.baseline_runner         import baseline_embedding
from helper_functions.feature_extraction.sequence_runner         import generateSequences, run_sequences
from helper_functions.feature_extraction.declaredevmining        import declare_embedding
from helper_functions.feature_extraction.ddm_newmethod_fixed_new import declare_data_aware_embedding
from helper_functions.feature_extraction.payload_extractor       import payload_embedding

# File utilities
from helper_functions.feature_extraction.PandaExpress            import dataframe_multiway_equijoin, fast_csv_parse, ensureDataFrameQuality
from helper_functions.feature_extraction.DumpUtils               import read_single_arff_dump, read_arff_embedding_dump, dump_custom_dataframes
from helper_functions.extra_functions                            import (
    build_conf,
    _make_value_eq,
    _make_occurrence,
    _make_sat_all_prop,
    read_sequence_log_via_arff,
    internal_to_folder,
    read_all_numeric_columns_except_one,
    read_feature_csv,
    dataframe_join_with_checks,
    dataframe_multiway_equijoin,
    multijoined_dump_no_splits,
    dump_all_compositions,
)

## 1) Tagging & Labeling

### 1.1) Retag raw log with unique identifiers

In [ ]:
# Assign stable case and event identifiers to the raw XES log
unique_log, log_obj = changeLog(RAW_LOG, output_dir=UNIQUE_DIR)

# Report output path
print(f"Retagged log written to: {unique_log}")

### 1.2) Build default configuration and define labelling strategies (Uncomment one per run based on which log is being processed)

#### TRAFFIC

In [ ]:
# Build a ConfigurationFile with shared settings for the Traffic experiments
def build_conf(exp_name: str) -> ConfigurationFile:
    cf = ConfigurationFile()
    cf.setExperimentName(exp_name)
    cf.setLogName(os.path.basename(unique_log))     # use the retagged filename
    cf.setOutputFolder("TRAFFIC")                   # base output subfolder
    cf.setMaxDepth(5)                               # decision-tree max depth
    cf.setMinLeaf(5)                                # minimum samples per leaf
    cf.setSequenceThreshold(SEQ_THRESHOLD)          # drop short sequences
    cf.setPayloadType(PayloadType.both)             # include control-flow and payload
    cf.setAutoIgnore([
        "time:timestamp", "concept: name", "Label", # ignore non-informative attributes
        "Start date", "End date", "Diagnosis",
        "Diagnosis code", "Diagnosis Treatment",
        "Combination ID", "Treatment code", "Activity code"
    ])
    cf.setPayloadSettings("traffic_settings.cfg")   # custom payload rules
    return cf

# Labeling functions used to annotate the log for different experiments
labeling_map = {
    "traffic_payload_Pay36": lambda log: tagLogWithValueEqOverEventAttn(
        log, "paymentAmount", 36.0
    ),
    "traffic_mr_tr": lambda log: tagLogWithOccurrence(
        log, ["Add penalty", "Payment"], 1
    ),
    "traffic_decl3": lambda log: tagLogWithSatAllProp(
        log,
        [(template_response, ["Insert Date Appeal to Prefecture", "Add penalty"])],
        SatCases.NotVacuitySat
    ),
}

#### SEPSIS

In [ ]:
# # Build a ConfigurationFile with shared settings for Sepsis experiments
# def build_conf(exp_name: str) -> ConfigurationFile:
#     cf = ConfigurationFile()
#     cf.setExperimentName(exp_name)
#     cf.setLogName(os.path.basename(unique_log))      # retagged filename
#     cf.setOutputFolder("SEPSIS")
#     cf.setMaxDepth(5)                                # decision-tree max depth
#     cf.setMinLeaf(5)                                 # minimum samples per leaf
#     cf.setSequenceThreshold(SEQ_THRESHOLD)           # drop short sequences
#     cf.setPayloadType(PayloadType.both)              # control-flow + payload
#     cf.setAutoIgnore([
#         "Diagnosis", "Diagnose",
#         "time:timestamp", "concept: name", "Label",
#         "lifecycle: transition",
#     ])
#     cf.setPayloadSettings("sepsis_settings.cfg")
#     return cf

# # Labeling functions for Sepsis experiments
# labeling_map = {
#     "sepsis_payload2": lambda log: tagLogWithValueEqOverIthEventAttn(
#         log, "DisfuncOrg", True, 0
#     ),
#     "sepsis_mr_tr": lambda log: tagLogWithExactOccurrence(
#         log, ["Admission NC", "CRP", "Leucocytes"], 1
#     ),
#     "sepsis_decl": lambda log: tagLogWithSatAllProp(
#         log,
#         [
#             (template_response, ["IV Antibiotics", "Leucocytes"]),
#             (template_response, ["LacticAcid", "IV Antibiotics"]),
#             (template_response, ["ER Triage", "CRP"]),
#         ],
#         SatCases.NotVacuitySat,
#     ),
# }


#### BPI15A

In [ ]:
# # Build a ConfigurationFile with shared settings for BPI15A experiments
# def build_conf(exp_name: str) -> ConfigurationFile:
#     cf = ConfigurationFile()
#     cf.setExperimentName(exp_name)
#     cf.setLogName(os.path.basename(unique_log))      # retagged filename
#     cf.setOutputFolder("BPI15A")
#     cf.setMaxDepth(5)                                # decision-tree max depth
#     cf.setMinLeaf(5)                                 # minimum samples per leaf
#     cf.setSequenceThreshold(SEQ_THRESHOLD)           # drop short sequences
#     cf.setPayloadType(PayloadType.both)              # control-flow + payload
#     cf.setAutoIgnore([
#         "time:timestamp", "concept: name", "Label",
#         "Start date", "End date", "Diagnosis",
#         "Diagnosis code", "Diagnosis Treatment",
#         "Combination ID", "Treatment code", "Activity code",
#         "dateFinished", "panned", "dueDate",
#     ])
#     cf.setPayloadSettings("bpi2015_settings.cfg")
#     return cf

# # Labeling functions for BPI15A experiments
# labeling_map = {
#     "bpi15A_payload_560925": lambda log: tagLogWithValueEqOverEventAttn(
#         log, "monitoringResource", "560925"
#     ),
#     "bpi15A_mr_tr": lambda log: tagLogWithOccurrence(
#         log, ["08_AWB45_005", "01_HOOFD_200"], 1
#     ),
#     "bpi15A_decl2": lambda log: tagLogWithSatAllProp(
#         log,
#         [(template_exist, ["01_HOOFD_011"])],
#         SatCases.NotVacuitySat,
#     ),
# }


### 1.3) Apply each labeling strategy and write out labeled logs

In [ ]:
# Run each labeling experiment and persist its outputs
for exp_name, label_func in labeling_map.items():
    print(f"\nProcessing label set: {exp_name}")

    # Build and save configuration for this experiment
    cf = build_conf(exp_name)
    json_path = os.path.join(LABELED_DIR, f"{exp_name}.json")
    cf.dump(json_path)

    # Apply labeling to the log
    tagger = TaggingStrategy(exp_name, label_func)
    tagger(LABELED_DIR, cf, log_obj)

    # Report generated artifacts
    xes_path = os.path.join(LABELED_DIR, tagger.logname)
    print(f"  Labeled XES:  {xes_path}")
    print(f"  Config JSON:  {json_path}")

## 2) Preparing Feature Extraction & Encoding

### 2.1) Prepare labeled log and output directory for feature extraction

In [ ]:
exp_name = cfg["experiment_name"]

# Paths to labeled log and its config
labeled_log = os.path.join(LABELED_DIR, f"{exp_name}.xes")
json_config = os.path.join(LABELED_DIR, f"{exp_name}.json")

# Output folder for extracted features
features_folder = os.path.join(FEATURES_DIR, f"{exp_name}_features")
os.makedirs(features_folder, exist_ok=True)

# Load labeled log
log = read_XES_log(labeled_log)
print(f"Loaded '{exp_name}': {len(log)} traces")

### 2.2) Generate propositional and canonical representations of the log

In [ ]:
# Collect trace identifiers for later joins and checks
trace_ids = {getTraceId(t) for t in log}

# Propositional representation for sequence/baseline models
propositional_log = xes_to_propositional_split(log)
print("Propositional traces ready")

# Propositional representation including event attributes (for DWD)
data_log = xes_to_data_propositional_split(log, doForce=False)
print("Data-propositional traces ready")

# Case-based list representation (used by specific embeddings)
trace_list = xes_to_tracelist_split(log)
print("Trace list ready")

## 3) Individual‐Activity Frequencies (Baseline Encoding)

In [ ]:
# Baseline (Individual-Activity Frequencies) features
baseline_folder = os.path.join(features_folder, "baseline")
os.makedirs(baseline_folder, exist_ok=True)

# Compute baseline embedding on the propositional log
processed_ids = baseline_embedding(
    baseline_folder,
    propositional_log,
    None
)

# Summary
print(f"Baseline features written to: {baseline_folder}")
print(f"Processed traces: {len(processed_ids)}")

## 4) Sequential Encoding (MR / TR / TRA / MRA)

In [ ]:
# Paths
log_file_name    = f"{exp_name}.xes"
sequence_folder  = os.path.join(features_folder, "sequence")
error_log_folder = os.path.join(sequence_folder, "errors")

# Ensure output directories exist
os.makedirs(sequence_folder, exist_ok=True)
os.makedirs(error_log_folder, exist_ok=True)

# Mine sequential strategies (n-gram–like patterns) from the labeled log
# Note: an alternative runner `run_sequences(...)` is available but not used here.
strategies = generateSequences(
    inp_path=LABELED_DIR,
    log_path=log_file_name,
    results_folder=sequence_folder,
    sequence_threshold=SEQ_THRESHOLD,
    err_logger=error_log_folder,
)

print("Mined strategies:", strategies)
print("Error logs (if any):", error_log_folder)

### 4.1) Reorganize strategy folders and clean up intermediate files

In [ ]:
# Promote sequence outputs from the intermediate split folder to the features root
split_folder = os.path.join(sequence_folder, "split1")

if os.path.isdir(split_folder):
    for strategy in os.listdir(split_folder):
        src = os.path.join(split_folder, strategy)
        dst = os.path.join(features_folder, strategy)

        # Replace any existing destination with the fresh result
        if os.path.exists(dst):
            shutil.rmtree(dst)

        shutil.move(src, dst)
        print(f"Moved {strategy} → {dst}")

    # Clean up the intermediate directory
    shutil.rmtree(sequence_folder)
    print(f"Removed intermediate folder: {sequence_folder}")
else:
    print(f"No split directory at {split_folder}; nothing to move.")

### 4.2) Write CSV for every individual sequential encoding

In [ ]:
encodings = ["mr", "mra", "tr", "tra"]

for enc in encodings:
    # Output folder for this encoding
    enc_folder = os.path.join(features_folder, enc)
    os.makedirs(enc_folder, exist_ok=True)

    # Load train/test splits and combine to a full set
    tr_df, te_df = read_sequence_log_via_arff(features_folder, enc)
    total_df = pd.concat([tr_df, te_df])

    # Output paths
    paths = {
        "total": os.path.join(enc_folder, f"{enc}.csv"),
        "train": os.path.join(enc_folder, f"{enc}_train.csv"),
        "test":  os.path.join(enc_folder, f"{enc}_test.csv"),
    }

    # Persist to CSV (semicolon separator expected downstream)
    total_df.to_csv(paths["total"], index=False, sep=";")
    tr_df.to_csv(paths["train"], index=False, sep=";")
    te_df.to_csv(paths["test"],  index=False, sep=";")

    # Log written files
    print(f"{enc}: wrote total → {paths['total']}")
    print(f"     train → {paths['train']}")
    print(f"     test  → {paths['test']}")

In [ ]:
# Sequence encodings to export
encodings = ["mr", "mra", "tr", "tra"]

for enc in encodings:
    enc_folder = os.path.join(features_folder, enc)
    os.makedirs(enc_folder, exist_ok=True)

    # Load train/test splits and combine to a full set
    tr_df, te_df = read_sequence_log_via_arff(features_folder, enc)
    total_df = pd.concat([tr_df, te_df])

    # Write combined split to CSV (semicolon delimiter expected downstream)
    total_path = os.path.join(enc_folder, f"{enc}.csv")
    total_df.to_csv(total_path, index=False, sep=";")

    print(f"{enc}: wrote total → {total_path}")

    # Optional: keep only the combined file
    # for file_path in glob.glob(os.path.join(enc_folder, "*.csv")):
    #     if file_path != total_path:
    #         os.remove(file_path)

### 4.3) Provide proper column naming (no collisions when merging)

In [ ]:
for enc in encodings:
    enc_folder = os.path.join(features_folder, enc)

    # Load activity code → name mapping
    mapping_path = os.path.join(enc_folder, "mapping.txt")
    mapping_df   = pd.read_csv(mapping_path, sep=";", header=0)
    code_to_name = dict(zip(mapping_df["activityCode"], mapping_df["activityName"]))

    # Rename feature columns in each split based on mapped activity names
    for split, suffix in [("total", ""), ("train", "_train"), ("test", "_test")]:
        csv_path = os.path.join(enc_folder, f"{enc}{suffix}.csv")
        if not os.path.exists(csv_path):
            continue  # split may not exist

        # Preserve delimiter consistency (files were written with ';')
        df = pd.read_csv(csv_path, sep=";")

        # Translate 3-char activity codes in column names to readable labels
        new_columns = []
        for col in df.columns:
            codes = [col[i:i+3] for i in range(0, len(col), 3)]
            names = [code_to_name[c] for c in codes if c in code_to_name]
            new_columns.append(f"{enc}[{', '.join(names)}]" if names else col)

        df.columns = new_columns
        df.to_csv(csv_path, index=False, sep=";")
        print(f"{enc}{suffix}.csv → columns renamed and saved")

## 5) Declarative Encoding (using Declare templates)

In [ ]:
# Declare features
declare_folder = os.path.join(features_folder, "declare")
os.makedirs(declare_folder, exist_ok=True)

# Compute Declare embedding on the full propositional log
processed_ids = declare_embedding(
    declare_folder,
    propositional_log,        # full log (no split)
    None,                     # placeholder for legacy API
    None,                     # no ExperimentRunner
    reencode=False,
    candidate_threshold=SPLIT_RATIO,
    constraint_threshold=SPLIT_RATIO,
)

# Summary
print(f"Declare features written to: {declare_folder}")
print(f"Total traces: {len(processed_ids)}")

## 6) Data-Aware Declare Encoding

In [ ]:
# Data-Aware Declare (DWD) features
dwd_folder = os.path.join(features_folder, "dwd")
os.makedirs(dwd_folder, exist_ok=True)

# Event attributes to exclude from data-aware constraints
ignored_attrs = [
    "time:timestamp", "concept: name", "Label", "Start date", "End date",
    "Diagnosis", "Diagnosis code", "Diagnosis Treatment", "Combination ID",
    "Treatment code", "Activity code",
]

# Compute DWD embedding on the full data-aware propositional log
processed_ids = declare_data_aware_embedding(
    ignored_attrs,
    dwd_folder,
    data_log,                 # full log (no split)
    None,                     # legacy placeholder
    missing_literal=None,
    self=None,                # in-memory runner (or a runner instance)
    candidate_threshold=SPLIT_RATIO,
    constraint_threshold=SPLIT_RATIO,
)

# Summary
print(f"Data-Aware Declare features written to: {dwd_folder}")
print(f"Total traces: {len(processed_ids)}")


## 7) Payload Encoding (Event & Trace Attribute Aggregations)

In [ ]:
# Payload features
payload_folder = os.path.join(features_folder, "payload")
os.makedirs(payload_folder, exist_ok=True)

# Path to payload configuration (empty for some logs, e.g., Traffic)
payload_settings = os.path.join(PROJECT_ROOT, "log_settings", EVENT_LOG_SETTINGS)
# payload_settings = ""  # optional override

processed_ids = payload_embedding(
    payload_folder,
    payload_settings,
    trace_list,
    None,
    None,
)

# Summary
print(f"Payload features written to: {payload_folder}")
print(f"Total traces: {len(processed_ids)}")

## 8) Feature concatenations

### 8.1) Combine sequential feature encodings across all strategies into 'sequential_combined'

In [ ]:
# Merge sequential encodings (mr, mra, tr, tra) into a single feature set per split
encodings = ["mr", "mra", "tr", "tra"]

sequential_combined_folder = os.path.join(features_folder, "seq_combined")
os.makedirs(sequential_combined_folder, exist_ok=True)

# Split suffixes used in filenames
splits = {"total": "", "train": "_train", "test": "_test"}

def load_and_merge(split_suffix: str) -> pd.DataFrame:
    """Load each encoding for a given split and merge on Case_ID."""
    dfs = []
    for i, enc in enumerate(encodings):
        csv_path = os.path.join(features_folder, enc, f"{enc}{split_suffix}.csv")
        df = pd.read_csv(csv_path, sep=";")
        # Keep a single Label column (from the first dataframe)
        if i > 0 and "Label" in df.columns:
            df = df.drop(columns=["Label"])
        dfs.append(df)

    merged = dfs[0]
    for df in dfs[1:]:
        merged = pd.merge(merged, df, on="Case_ID", how="outer")
    return merged

for split, suffix in splits.items():
    combined_df = load_and_merge(suffix)
    out_path = os.path.join(sequential_combined_folder, f"seq_combined{suffix}.csv")
    combined_df.to_csv(out_path, index=False, sep=";")
    print(f"Wrote: {out_path}")


### 8.2) Make sure all Case_ID's are set to index

In [ ]:
# Set Case_ID as the index for each strategy's combined CSV
strategies = [d for d in os.listdir(features_folder)
              if os.path.isdir(os.path.join(features_folder, d))]

for strat in strategies:
    csv_path = os.path.join(features_folder, strat, f"{strat}.csv")
    if not os.path.exists(csv_path):
        continue  # skip folders without a combined CSV

    # Files in this pipeline use semicolon as delimiter
    df = pd.read_csv(csv_path, sep=";")
    df.set_index("Case_ID", inplace=True)
    df.to_csv(csv_path, sep=";")

    print(f"Set index for {strat} → {csv_path}")

### 8.3) Concatenating new combinations of extracted features

In [ ]:
dataset_composition = cfg["dataset_composition"]
folder_to_internal = cfg["folder_to_internal"]
internal_to_folder_map = internal_to_folder(folder_to_internal)

In [ ]:
# to join a single composition:
# joined_df = multijoined_dump_no_splits(
#     key="baseline_data",
#     dataset_list=dataset_composition["baseline_data"],
#     base_folder=features_folder,
#     folder_to_internal_map=folder_to_internal
# )

# or to dump all:
dump_all_compositions(
    dataset_composition,
    base_folder=features_folder,
    folder_to_internal_map=folder_to_internal
)

## 9) Some final checks

In [ ]:
# Ensure each strategy's CSV uses Case_ID as the index
strategies = [
    d for d in os.listdir(features_folder)
    if os.path.isdir(os.path.join(features_folder, d))
]

for strat in strategies:
    csv_path = os.path.join(features_folder, strat, f"{strat}.csv")
    if not os.path.exists(csv_path):
        continue

    # Files in this pipeline use a semicolon delimiter
    df = pd.read_csv(csv_path, sep=";")
    df.set_index("Case_ID", inplace=True)
    df.to_csv(csv_path, sep=";")

    print(f"Set index for {strat} → {csv_path}")